In [1]:
# pandas is a python library for data manipulation and analysis
import pandas as pd
# numpy is a python library that supports multidimensional array and matrix manipulation
import numpy as np

# Load the data from the csv into a dataframe
df = pd.read_csv('word_sentiments.csv')
# Drop unnecessary column (need inplace otherwise it doesn't save the changes to df)
df.drop('Unnamed: 0', axis='columns', inplace=True)


In [7]:
# This is what the first 5 rows of our dataframe looks like
df.head()


,word,score,embedding
0,is,0.125,"[0.0070495605, -0.07324219, 0.171875, 0.022583..."
1,not,-0.625,"[0.08496094, -0.095214844, 0.119140625, 0.1118..."
2,will,0.125,"[0.048828125, 0.16699219, 0.16894531, 0.087402..."
3,an,-0.125,"[0.12597656, 0.19042969, 0.06982422, 0.0722656..."
4,had,0.250,"[-0.05810547, 0.05810547, 0.013305664, -0.0003..."


In [2]:
from gensim.models import KeyedVectors

# load the word embeddings into the model from file
filename = 'word2vecSmall.bin.gz'
model = KeyedVectors.load_word2vec_format(filename, binary=True)

# Add column for word embeddings to dataframe
# lambda functions are functions without names - what this is essentially saying is to go through
# every word in our dataframe and access its word embedding from the model
df['embedding'] = df['word'].apply(lambda x: model[x])


In [3]:
# These are the words we want to test our model with - feel free to change these!
# Ask audience to give us these words
test_words = ['nice', 'mean', 'bad', 'good', 'sad', 'happy', 'fantastic', 'terrible']


In [4]:
# This is the data we are testing our model on (contains words in blacklist)
# This is selecting all the rows in the dataframe whose rows are in the list of test words above, ignoring
# uppercase/lowercase.
test_df = df[df['word'].str.lower().str.strip().isin(test_words)].copy()
test_df


,word,score,embedding
16,good,0.500,"[0.040527344, 0.0625, -0.017456055, 0.07861328..."
106,bad,-0.875,"[0.06298828, 0.12451172, 0.11328125, 0.0732421..."
164,happy,0.875,"[-0.0005187988, 0.16015625, 0.0016098022, 0.02..."
649,Good,0.500,"[-0.10888672, -0.07470703, -0.045410156, -0.00..."
717,fantastic,0.375,"[-0.122558594, -0.037841797, -0.12402344, 0.02..."
750,sad,-0.625,"[0.18945312, 0.045898438, 0.06689453, -0.04467..."
907,terrible,-0.625,"[0.1640625, 0.19238281, 0.092285156, 0.1308593..."
1833,Bad,-0.875,"[-0.078125, -0.11279297, 0.018676758, 0.080566..."
2288,Happy,0.875,"[0.05078125, -0.109375, -0.12597656, 0.1240234..."
2638,GOOD,0.500,"[-0.34179688, -0.41015625, 0.45117188, -0.2871..."


In [5]:
# This is the data we are training our model on (words NOT in blacklist - that's what the tilda does)
train_df = df[~df['word'].str.lower().str.strip().isin(test_words)].copy()
train_df.head()


NameError: name 'blacklist' is not defined

In [ ]:
# Ensures input to LinearRegression model is in acceptable format
# stack the word embeddings into a matrix of size (# words, len word embedding)
X_train = np.stack(train_df['embedding'])

# get sentiment scores of training words
Y_train = train_df['score'].values


In [ ]:
print(X_train.shape)
print(Y_train.shape)


In [ ]:
X_test = np.stack(test_df['embedding'])
Y_test = test_df['score'].values


In [ ]:
print(X_test.shape)
print(Y_test.shape)


In [ ]:
# Need to import model from sci-kit-learn, a popular python machine learning library
from sklearn.linear_model import LinearRegression

classifier = LinearRegression()

# Training step that Jason discussed - find optimal coefficients m1, m2, ... m300
# and intercept b for all elements in word embedding to predict the sentiment score.
classifier.fit(X_train, Y_train)


In [ ]:
# This is the m1, m2, ..., m300 we wanted to find.
classifier.coef_


In [ ]:
# This gets the shape of coefficients of the LinearRegression model, which we can see is the same
# as the length of the embedding input.
classifier.coef_.shape


In [ ]:
# This is the intercept of the LinearRegression model (the 'b' in y = mx + b)
classifier.intercept_


In [ ]:
# Gets the r^2 coefficient - don't need to know nitty gritty of this, it just measures how
# accurate our model is.
# We want our r^2 to be above 0 (positive correlation between predicted sentiment scores and actual scores)
# and as close to 1 as possible (that would mean perfect correlation).

# 0.58 is pretty good!
print("Score:", classifier.score(X_test, Y_test))


In [ ]:
# To see this in action, let's get a word from our test set that obviously has a positive sentiment and
# one with an obvious negative sentiment
happy_vector = [np.array(model['happy'])]
sad_vector = [np.array(model['sad'])]


In [ ]:
# Here's an example of output from the model - 'happy' should have a higher sentiment score than 'sad'
print(classifier.predict(happy_vector))
print(classifier.predict(sad_vector))


In [ ]:
# It would help us humans understand what's going on better if we had a way to visualize these predictions.
# pandas has some pretty nice built in plotting features - we're going to put the predicted scores
# into the dataframe so pandas can plot it for us.
#
# We can create a new column called predicted_score and set it equal to the classifier's output for the X_test
# matrix we made above.
test_df['predicted_score'] = classifier.predict(X_test)

# To make our plot prettier let's sort by predicted scores, so bars will be displayed in order of 
# lowest to highest predicted sentiment
test_df.sort_values(by=['predicted_score'],inplace=True)


In [ ]:
# We can now see the new column we've added into the dataframe.
test_df


In [ ]:
# A bar plot showing what our model thinks the sentiments of our test words are and what they actually are
# Doesn't get the magnitude of positive or negative sentiment that well, but it gets the general idea!
test_df.plot.bar(x='word',y=['predicted_score', 'score'])
